# Esercizio 3 — GroupBy e trasformazioni (propedeutico)

**Obiettivi:**
- Usare `groupby` e `transform` per ottenere statistiche di gruppo e applicarle riga-per-riga;
- Creare colonne che mostrino differenze rispetto alla media di gruppo;
- Ordinare i risultati per interpretazione.


## Setup: import e `DataFrame`
`pd.DataFrame({...})` costruisce la tabella a partire da un dizionario di liste. Importiamo `pandas as pd` per usare l'alias `pd`. La cella termina con `df`, quindi Jupyter ne visualizza il contenuto.


In [ ]:
import pandas as pd

df = pd.DataFrame({
    "Giorno": ["Lun","Mar","Mer","Gio","Ven","Sab","Dom"],
    "Temperatura": [18,20,21,19,23,25,22],
    "Umidita": [60,55,58,63,50,45,52],
    "Pioggia": [0,1,0,1,0,0,1],
    "Vento": [5,12,8,20,7,6,15]
})

df

## Media della temperatura per gruppi di pioggia
- `df.groupby("Pioggia")` raggruppa le righe in base al valore della colonna `Pioggia` (0 = no pioggia, 1 = pioggia).
- `[...] ["Temperatura"]` seleziona la colonna su cui calcolare la statistica.
- `.transform("mean")` applica la media a ciascun gruppo ma restituisce una serie lunga quanto l'intero `DataFrame`, così ogni riga riceve la media del proprio gruppo (diversamente da `.agg`, che ridurrebbe il numero di righe).


In [ ]:
df_tempmedia = df.groupby("Pioggia")["Temperatura"].transform("mean")
df_tempmedia

,Giorno,Temperatura,Umidita,Pioggia,Vento,Temp_Delta_Pioggia
0,Lun,18,60,0,5,-3.750000
1,Mar,20,55,1,12,-0.333333
2,Mer,21,58,0,8,NaN
3,Gio,19,63,1,20,NaN
4,Ven,23,50,0,7,NaN
5,Sab,25,45,0,6,NaN
6,Dom,22,52,1,15,NaN


## Differenza tra temperatura e media di gruppo
Creiamo `Temp_Delta_Pioggia` come differenza tra la temperatura della riga e la media del suo gruppo di pioggia.


In [9]:
df["Temp_Delta_Pioggia"] = df["Temperatura"] - df_tempmedia
df

,Giorno,Temperatura,Umidita,Pioggia,Vento,Temp_Delta_Pioggia
0,Lun,18,60,0,5,-3.750000
1,Mar,20,55,1,12,-0.333333
2,Mer,21,58,0,8,-0.750000
3,Gio,19,63,1,20,-1.333333
4,Ven,23,50,0,7,1.250000
5,Sab,25,45,0,6,3.250000
6,Dom,22,52,1,15,1.666667


## Differenza tra temperatura e media di gruppo
`df["Temperatura"] - df_tempmedia` sfrutta il fatto che `df_tempmedia` ha la stessa lunghezza di `df`: la sottrazione è elemento-per-elemento e restituisce l'anomalia della temperatura rispetto al gruppo di pioggia. Assegnando il risultato a `df["Temp_Delta_Pioggia"]` creiamo una nuova colonna.


In [10]:
df.sort_values("Temp_Delta_Pioggia", ascending= False )

,Giorno,Temperatura,Umidita,Pioggia,Vento,Temp_Delta_Pioggia
5,Sab,25,45,0,6,3.250000
6,Dom,22,52,1,15,1.666667
4,Ven,23,50,0,7,1.250000
1,Mar,20,55,1,12,-0.333333
2,Mer,21,58,0,8,-0.750000
3,Gio,19,63,1,20,-1.333333
0,Lun,18,60,0,5,-3.750000


## Riepilogo funzioni e concetti (GroupBy e Transform)
Spiegazione puntuale degli strumenti usati.

### `groupby`
- `df.groupby('Pioggia')`: crea un oggetto gruppo che separa le righe per valore della colonna `Pioggia`. Ogni gruppo è elaborato separatamente.
- Con più colonne: `df.groupby(['Pioggia','Vento_Class'])` (esempio concettuale).

### Selezione post-groupby
- `df.groupby('Pioggia')['Temperatura']`: seleziona la colonna su cui calcolare statistiche (riduce il focus ai valori della variabile `Temperatura` all'interno dei gruppi).

### `transform('mean')` vs `agg('mean')`
- `agg('mean')`: restituisce un risultato compatto (una riga per gruppo).
- `transform('mean')`: espande il risultato riportando per ogni riga la statistica del suo gruppo (lunghezza invariata). Utile per confronti riga‑gruppo.

### Creazione di colonne derivate
- `df['Temp_Delta_Pioggia'] = df['Temperatura'] - media_di_gruppo`: differenza tra il valore individuale e la media del gruppo → misura quanto una riga si discosta dal comportamento medio.

### Ordinamento
- `df.sort_values('Temp_Delta_Pioggia', ascending=False)`: mostra prima le righe con maggiore scostamento positivo (giorni più caldi rispetto alla media del loro stato di pioggia).

### Interpretazione del delta
- Valore > 0: sopra la media del gruppo.
- Valore < 0: sotto la media.
- Valore ≈ 0: in linea con la media.

### Vantaggi di `transform`
- Permette di combinare facilmente statistiche di gruppo con dati originali senza join manuali.
- Mantiene l'allineamento degli indici.

### Buone pratiche
- Nomina chiaramente le colonne derivate (`*_Delta_*`, `*_Ratio_*`).
- Usa `sort_values` per facilitare l'interpretazione visiva.
- Evita calcoli ripetuti: assegna una volta la serie trasformata se la riusi.

Con questi punti ogni passaggio del notebook risulta motivato e leggibile.
